In [12]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys
from collections import defaultdict
import urllib
from bs4 import BeautifulSoup
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from textblob import TextBlob
#import jsonpickle

In [2]:
class MyModelParser(tweepy.parsers.ModelParser):
    def parse(self, method, payload):
        result = super(MyModelParser, self).parse(method, payload)
        result._payload = json.loads(payload)
        return result

In [3]:
def load_api():
    ''' Function that loads the twitter API after authorizing the user. '''

    consumer_key = 'qiOaRLVPldazXmYmF3IaIQw4L'
    consumer_secret = 'FAzcCMF1UUyuNuSeddAA1nDJYPeXm6OhaCD084k1t3BZ0HleCY'
    access_token = '717220472623071233-oeDgXLyYdqT92Mi06aaAGV7EtSExfKS'
    access_secret = 'tVh6WAqVqJ5Pekb3skPON4OD46dyyBAIGOiWjPkZrtglC'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)


In [4]:
def tweet_search(api, query, max_tweets, max_id, since_id):
    ''' Function that takes in a search string 'query', the maximum
        number of tweets 'max_tweets', and the minimum (i.e., starting)
        tweet id. It returns a list of tweepy.models.Status objects. '''

    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets,
                                    since_id=str(since_id),
                                    max_id=str(max_id-1))
#                                    geocode=geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id

In [5]:
tweets_data = []
api = load_api()
search_phrases = ['#edpolicy','#edreform', 
                     '#edreformtribe', '#edgap',
                     '#literacy', '#teacherquality',
                     '#urbaned', 'edadmin',
                     '#edpolitics']
maxTweets = 100

In [6]:
for search_phrase in search_phrases:
    for tweets in tweepy.Cursor(api.search,search_phrase,\
                           lang="en",\
                           since='2014-06-06',until='2017-06-06').items(maxTweets):
        #print tweet.created_at, tweet.text
    #print type(tweets)
        tweets_data.append(tweets)
    #csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

In [7]:
json_tweet_data = []
for tweets in tweets_data:
    json_tweet_data.append(tweets._json)

In [8]:
print len(tweets_data)
print len(json_tweet_data)

486
486


In [9]:
stanford_ner_dir = '/home/dell/stanford-ner-2015-04-20/'
eng_model_filename= stanford_ner_dir + 'classifiers/english.all.3class.distsim.crf.ser.gz'
my_path_to_jar= stanford_ner_dir + 'stanford-ner.jar'

st = StanfordNERTagger(model_filename=eng_model_filename, path_to_jar=my_path_to_jar)
enc = lambda x: x.encode('latin1', errors='ignore')

In [10]:
tweets_dict = defaultdict(list)
for tweets in json_tweet_data:
    if tweets.has_key('user'):
        #print 'No user data - ignoring tweet.'
        user = enc(tweets['user']['name'])
        text = enc(tweets['text'])
        #url = re.search("(?P<url>https?://[^\s]+)", text.lower())
        expanded_url = tweets['entities']['urls']
        #print expanded_url
        if len(expanded_url)!=0 and expanded_url[0]['expanded_url']:
            url = expanded_url[0]['expanded_url']
            content = urllib.urlopen(url).read()
            soup = BeautifulSoup(content)
            tokenized_text = word_tokenize(soup.text)
            classified_text = st.tag(tokenized_text)
            for term,tag in classified_text:
                if tag == 'LOCATION':
                    if term in tweets_dict.keys():
                        tweets_dict[term].append(soup.text)
                    else:
                        tweets_dict[term]=[]
                        tweets_dict[term].append(soup.text)
        
        else:
            tokenized_text = word_tokenize(text)
            classified_text = st.tag(tokenized_text)
            for term,tag in classified_text:
                if tag == 'LOCATION':
                    if term in tweets_dict.keys():
                        tweets_dict[term].append(text)
                    else:
                        tweets_dict[term]=[]
                        tweets_dict[term].append(text)

/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/dell/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [13]:
positive = defaultdict(list)
negative = defaultdict(list)
neutral = defaultdict(list)
for keys in tweets_dict.keys():
    for tweets in tweets_dict[keys]:
        # pass tweet into TextBlob
        tweet = TextBlob(tweets)

        # output sentiment polarity
        #print keys
        #print tweet.sentiment.polarity

        # determine if sentiment is positive, negative, or neutral
        if tweet.sentiment.polarity < 0:
            #sentiment = "negative"
            if keys in negative.keys():
                negative[keys].append(tweets)
            else:
                negative[keys]=[]
                negative[keys].append(tweets)
        elif tweet.sentiment.polarity == 0:
            #sentiment = "neutral"
            if keys in neutral.keys():
                neutral[keys].append(tweets)
            else:
                neutral[keys]=[]
                neutral[keys].append(tweets)
        else:
            #sentiment = "positive"
            if keys in positive.keys():
                positive[keys].append(tweets)
            else:
                positive[keys]=[]
                positive[keys].append(tweets)

In [19]:
print "positive"+":"+str(len(positive.keys())) 
print "negative"+":"+str(len(negative.keys()))
print "neutral"+":"+str(len(neutral.keys()))
print positive.keys()

positive:280
negative:177
neutral:2
[u'Canada', u'JFK', u'Mississippi', u'United', u'Arial', u'WA', u'Bridges', u'Paris', u'Arlington', u'N.W', u'Missouri', u'Helvetica', u'Minnesota', u'Cambodia', u'Illinois', u'Newark', u'Johannesburg', u'Hill', u'Fourth', u'Indiana', u'Louisiana', u'Worth', u'Texas', u'U.S.Leyla', u'Santiago', u'Columbia', u'Janeiro', u'Sacramento', u'Kansas', u'PHILADELPHIA', u'Connecticut', u'Montana', u'Flint', u'York', u'World', u'Atlanta', u'Normaalikoulu', u'Haven', u'Netherlands', u'COLORADO', u'Berlin', u'Ventures', u'Island', u'de', u'Lexington', u'Crystal', u'Voila', u'Alexandria', u'Topeka', u'Chaika', u'V.P', u'Cincinnati', u'River', u'Jersey', u'Goshorn', u'Bolivar', u'BrooklynMareesa', u'England', u'San', u'D.C', u'Facility', u'Baltimore', u'Fresno', u'India', u'Kong', u'Nottingham', u'Durham', u'Dakota', u'Seattle', u'Lonely', u'CrossRidge', u'Rochester', u'Ireland', u'District', u'Southampton', u'Lake', u'Sunnyside', u'Hospital', u'Raleway', u'School